In [ ]:
# OrbitCOM, using Template by G. Besla & R. Li
# A function that calculates the orbits of particles up to snapshot 800 (~12 Gyr)
# Ryan Lewis
# 3/6/2020

In [ ]:
import numpy as np
import astropy.units as u
from astropy.constants import G

# import plotting modules
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# my modules
from ReadFile import Read
# Step 1: modify CenterOfMass so that COM_P now takes a parameter specifying 
# by how much to decrease RMAX instead of a factor of 2
from CenterOfMass2 import CenterOfMass

In [ ]:
# A function that computes the time and COM position and velocity vectors of a given galaxy
# in each snapshot and saves that output into a file
def OrbitCOM(Galaxy, Start, End, n):
    # Input:
        # Galaxy - the name of the galaxy (e.g. MW for Milky Way)
        # Start - the number of the first snapshot to be read in
        # End - the number of the last snapshot to be read in
        # n - an integer indicating the intervals over which you will return the COM
    # Output:
        # a file with computed time and COM position and velocity vectors of a given galaxy
    
    fileout = "Orbit_"+ galaxy + ".txt" # compose the filename for output
    
    # Set tolerance and VolDec for calculating COM_P in CenterOfMass
    # for M33 that is stripped more, use different values for VolDec
    delta = 0.1
    
    if galaxy == "M33":
        VolDec = 4.0
    else:
        VolDec = 2.0
    
    # Generate the snapshot id sequence 
    # It is always a good idea to also check if the input is eligible (not required)
    snap_ids = np.arange(start, end, n)
    
    if len(snap_ids) == 0: # Check if the input is eligible - if the array is empty
        print("Array is ineligible")
        return
    
    # initialize the array for orbital info: t, x, y, z, vx, vy, vz of COM
    orbit = np.zeros(len(snap_ids), 7)
    
    # a for loop 
    for i, snap_id in enumerate(snap_ids): # loop over files
        
        # compose the data filename (be careful about the folder)
        ilbl = '000' + str(Snap) # Add a string of the filenumber to the value "000"
        ilbl = ilbl[-3:] # remove all but the last 3 digits
        filename = "%s_"%(Galaxy) + ilbl + ".txt"
        
        # Initialize an instance of CenterOfMass class, using disk particles (2)
        COM = CenterOfMass(filename, 2)
        
        # Store the COM pos and vel. Remember that now COM_P required VolDec
        COMP = COM.COM_P(delta, VolDec)
        COMV = COM.COM_V(COMP[0], COMP[1], COMP[2])
    
        # store the time, pos, vel in ith element of the orbit array,  without units (.value) 
        # note that you can store 
        # a[i] = var1, *tuple(array1)
        orbit[i] = [snap_ids, COMP[0], COMP[1], COMP[2], COMV[0], COMV[1], COMV[2]]
        
        # print snap_id to see the progress
        print(snap_ids)
        
    # write the data to a file
    # we do this because we don't want to have to repeat this process 
    # this code should only have to be called once per galaxy.
    np.savetxt(fileout, orbit, fmt = "%11.3f"*7, comments='#',
               header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                      .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))

In [ ]:
# Recover the orbits and generate the COM files for each galaxy
# read in 800 snapshots in intervals of n=5
# Note: This might take a little while - test your code with a smaller number of snapshots first!
MW = OrbitCOM("MW", 0, 801, 5)
M31 = OrbitCOM("M31", 0, 801, 5)
M33 = OrbitCOM("M33", 0, 801, 5)

In [ ]:
# Read in the data files for the orbits of each galaxy that you just created
# headers:  t, x, y, z, vx, vy, vz
# using np.genfromtxt
MWData = np.genfromtxt("Orbit_MW.txt")
M31Data = np.genfromtxt("Orbit_M31.txt")
M33Data = np.genfromtxt("Orbit_M33.txt")

In [ ]:
# function to compute the magnitude of the difference between two vectors 
# You can use this function to return both the relative position and relative velocity for two 
# galaxies over the entire orbit
def MagDiff(v1, v2):
    # Input:
        # v1 - arbitary 3D vector
        # v2 - arbitrary vector of same dimension as v1
    # Output:
        # vsub - the magntiude of the difference between v1 and v2
        
    vsub = np.sqrt((v1[0]-v2[0])**2 + (v1[1]-v2[1])**2 + (v1[2]-v2[2])**2)
    
    return vsub

In [ ]:
# Determine the magnitude of the relative position and velocities

# of MW and M31
MW_M31_Diff = MagDiff(MWData, M31Data)
MWM31P = (MW_M31_Diff[:, 1], MW_M31_Diff[:, 2], MW_M31_Diff[:, 3])
MWM31V = (MW_M31_Diff[:, 4], MW_M31_Diff[:, 5], MW_M31_Diff[:, 6])

# of M33 and M31
M33_M31_Diff = MagDiff(M33Data, M31Data)
M33M31P = (M33_M31_Diff[:, 1], M33_M31_Diff[:, 2], M33_M31_Diff[:, 3])
M33M31V = (M33_M31_Diff[:, 4], M33_M31_Diff[:, 5], M33_M31_Diff[:, 6])


In [ ]:
# Plot the Orbit of the galaxies

fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)

plt.plot(MWData['t'], MWM31P, color='indianred', linewidth=4, label='M31 Orbit Around MW')
plt.plot(M31Data['t'], M33M31P, ':', color='mediumaquamarine', linewidth=4, label='M33 Orbit Around M31')

plt.xlabel('Time (Gyr)', fontsize=22)
plt.ylabel('Separation (kpc)', fontsize=22)

plt.legend(loc='upper right')
plt.show()

In [ ]:
# Plot the orbital velocities of the galaxies
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)

plt.plot(MWData['t'], MWM31P, color='indianred', linewidth=4, label='M31 Orbit Around MW')
plt.plot(M31Data['t'], M33M31P, '--', color='mediumaquamarine', linewidth=4, label='M33 Orbit Around M31')

plt.xlabel('Time (Gyr)', fontsize=22)
plt.ylabel('Separation (kpc)', fontsize=22)

plt.legend(loc='upper right')
plt.show()

In [ ]:
# Questions

# 1) How many close encounters will the MW and M31 experience in the future?
    # 
    
# 2) How is the time evolution of the separation and relative velocity related?
    # 

# 3) When do M31 and the MW merge? (you might need to zoom in on the plot - try a log y axis). 
#    What happens to M33’s orbit when they merge?

# BONUS: What is roughly the decay rate of M33’s orbit after 6 Gyr (ratio of the difference 
#        between two successive apocenters and the orbital period; you don’t need to be precise). 
#        If this rate is constant, how long will it take M33 to merge with the combined MW+M31 remnant if it 
#        is at a distance of 75 kpc?